# Предсказание оклада

## Вы научитесь:

• использовать линейную регрессию

• применять линейную регрессию к текстовым данным

## Введение

Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

## Реализация в Scikit-Learn

Загрузим данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv:

In [14]:
import pandas as pd
data_train=pd.read_csv('salary-train.csv')
data_test=pd.read_csv('salary-test-mini.csv')

Проведём предобработку данных:

Для начала преобразуем всё, кроме букв и цифр, на пробелы  — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

In [15]:
data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True,inplace=True) # в нижний регистр
data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True,inplace=True) # в нижний регистр

Применим  TfidfVectorizer для преобразования текстов в векторы признаков:

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5)
fullDescription_train=vectorizer.fit_transform(data_train['FullDescription'])#сбор статистики и трансформация характетистик в числа
fullDescription_test=vectorizer.transform(data_test['FullDescription'])#трансформация на основе ранее собранной статистики

Заменим пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan':

In [17]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

Применим DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime:

In [18]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объединим все полученные признаки в одну матрицу "объекты-признаки". Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack:

In [19]:
from scipy.sparse import hstack
joinedMatrix_train=hstack([fullDescription_train,X_train_categ])

Обучим гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized:

In [20]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0,random_state=241)
clf.fit(joinedMatrix_train, data_train['SalaryNormalized'])

Ridge(random_state=241)

Построим прогнозы для двух примеров из файла salary-test-mini.csv:

In [22]:
joinedMatrix_test=hstack([fullDescription_test,X_test_categ])#соединяем признаки
predict=clf.predict(joinedMatrix_test)

print('Прогноз по зарплате:', ' '.join([str(round(i,2)) for i in predict]))

Прогноз по зарплате: 56586.63 37120.93
